In [1]:
import json
import pandas as pd
import numpy as np
from skimage import io
from skimage import img_as_ubyte
from skimage import color
import util

In [2]:
REQUEST = json.dumps({
'path' : {},
'args' : {}
})

In [3]:
# ## 富士山の座標
# z="13"
# x="7252"
# y="3234"

# tellusAPIの画像を取得するだけのAPI

In [4]:
# GET /img/:img_type/:z/:x/:y

# 読み取り
request = json.loads(REQUEST)

# 必要項目抽出
img_type = request['path'].get('img_type')
x = request['path'].get('x')
y = request['path'].get('y')
z = request['path'].get('z')

if util.input_validation_check(x, y, z, img_type):
    img_np = util.get_image_using_tellus_api(x, y, z, img_type)
    if len(img_np) != 0:
        img_base64 = util.make_base64_image(img_np)
        print('<html><img src="data:image/png;base64, {}" /></html>'.format(img_base64))
    else:
        pass

ERROR: Input format error. please input `/img/[img_type]/[z]/[x]/[y]`
ex. http://localhost:8822/img/osm/13/7252/3234


In [5]:
# ResponseInfo GET /img/:img_type/:z/:x/:y
print(json.dumps({"headers" : {"Content-Type" : "text/html"},"status" : 201}))

{"headers": {"Content-Type": "text/html"}, "status": 201}


# NDSI画像を返すAPI

In [6]:
def ditect_snow(img):
    # グレイ変換
    img_gray_01 = color.rgb2gray(img)
    # 値のレンジを変更(0-1 -> 0-255)
    img_gray = img_as_ubyte(img_gray_01)

    # しきい値処理
    height, width= img_gray.shape
    img_binary = np.zeros((height, width)) # 初期化
    img_binary = img_gray < 230 # しきい値にもとづいて，2値化処理実行

    #雪だけをマスキング
    img_msk_rev = img.copy()
    img_msk_rev[~img_binary] = 40
    return img_msk_rev, img_binary

In [7]:
def ditect_ndsi(img_band2, img_band4):
    # NDSI（Normalized Difference Snow Index）を計算
    img_NDSI = (img_band2 - img_band4) / (img_band2 + img_band4)  # band4ではなく5を使わなくてはいけない？

    # 最大最小を制限
    img_NDSI = np.clip(img_NDSI, 0, 1)

    return img_NDSI

In [8]:
# GET /ndsi_img/:z/:x/:y

# 読み取り
request = json.loads(REQUEST)

# 必要項目抽出
x = request['path'].get('x')
y = request['path'].get('y')
z = request['path'].get('z')
# z="13"
# x="7252"
# y="3234"

if util.input_validation_check(x, y, z):
#     try:        
    imgs = util.get_images_using_tellus_api(x, y, z)
    img_natural = imgs[1]
    img_band2 = imgs[4]
    img_band4 = imgs[6]

    # 変換処理
    img_msk_rev, img_binary = ditect_snow(img_natural)

    # #雪以外をマスキングした場合
    img_NDSI = ditect_ndsi(img_band2, img_band4)
    img_NDSI[img_binary] = 0

    # 表示のために型変換
    img_base64 = util.make_base64_image(img_NDSI)

    # 出力
    print('<html><img src="data:image/png;base64, {}" /></html>'.format(img_base64))

#     except:
#         pass

ERROR: Input format error. please input `/img/[img_type]/[z]/[x]/[y]`
ex. http://localhost:8822/img/osm/13/7252/3234


In [9]:
# ResponseInfo GET  /ndsi_img/:z/:x/:y
print(json.dumps({"headers" : {"Content-Type" : "text/html"},"status" : 201}))

{"headers": {"Content-Type": "text/html"}, "status": 201}
